<h1> This file annotates NERs in tweets using a dataset of names and teams. </h1>

In [2]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from csv import reader
from csv import writer
import pandas as pd

In [3]:
def binarySearch(arr, l, r, x): 
    x = x.lower()
    while l <= r: 
        mid = l + (r - l)//2; 
        if arr[mid] == x: 
            return True 
        elif arr[mid] < x: 
            l = mid + 1
        else: 
            r = mid - 1
    return False

In [4]:
inittagsteam = []
inittagsplayer = []

<h3> This function takes in a csv file and a list and converts all elements to a list </h3>

In [5]:
def readTags(s,inittags):
    with open(s,'r') as readobj:
        csvread = reader(readobj)
        for row in csvread:
            for el in row:
                inittags.append(el)


In [6]:
readTags('nation.csv',inittagsteam)
readTags('clubs.csv',inittagsteam)
readTags('popnames.csv',inittagsplayer)
readTags('playername.csv',inittagsplayer)
readTags('shirtname.csv',inittagsplayer)
notflattagteam = []
notflattagplayer = []
for tag in inittagsteam:
    tagspace = tag.split()
    notflattagteam.append(tagspace)
for tag in inittagsplayer:
    tagspace = tag.split()
    notflattagplayer.append(tagspace)

<h3> Here we remove all NERs below 3 letters, and convert all to lower case. Two final lists are playertags and teamtags </h3>

In [7]:
teamtags = []
playertags = []
for sublist in notflattagteam:
    for item in sublist:
        if(len(item)<=2):
            continue
        teamtags.append(item.lower())
for sublist in notflattagplayer:
    for item in sublist:
        if(len(item)<=2):
            continue
        playertags.append(item.lower())
teamtags.sort()
playertags.sort()
teamtags = list(dict.fromkeys(teamtags) )
playertags = list(dict.fromkeys(playertags) )

In [8]:
df = pd.read_csv("FIFA.csv", usecols = ['Tweet'])
taggedsents = []
for sent in df['Tweet']:
    sent = word_tokenize(str(sent))
    sent = pos_tag(sent)
    taggedsents.append(sent)

<h2> Here is where we write to csv file. You can't open more than 1mil rows on most csv readers, but it should be fine and have around 4mil rows anyway. <h2>


In [9]:
seriallist, sentidlist, wordlist, postaglist, nertaglist = [],[],[],[],[]
flagteam, flagplayer = False, False
serial = 0
sentid = 0
for sentence in taggedsents:
    flagteam, flagplayer = False, False
    sentid += 1
    for wordtagpair in sentence:
        serial += 1
        word = wordtagpair[0]
        tag = wordtagpair[1]
        sentidlist.append(sentid)
        seriallist.append(serial)
        wordlist.append(word)
        postaglist.append(tag)
        if binarySearch(playertags,0,len(playertags),word):
            if flagplayer == True:
                nertaglist.append('INS-PLAY')
            else:
                nertaglist.append('BEG-PLAY')
                flagplayer = True
                flagteam = False
        elif binarySearch(teamtags,0,len(teamtags),word):
            if flagteam == True:
                nertaglist.append('INS-TEAM')
            else:
                nertaglist.append('BEG-TEAM')
                flagteam = True
                flagplayer = False
        else:
            nertaglist.append('OUT')
            flagteam, flagplayer = False, False
data = {'Serial Number':seriallist,
        'Sentence ID':sentidlist,
        'Word':wordlist,
        'POS Tag':postaglist,
         'NER Tag':nertaglist}
df = pd.DataFrame(data)
df.to_csv(r'dataset.csv', index = False, header=True)